In [1]:
import time, re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [2]:
def initialize_driver(executable_path):

    s = Service(executable_path=executable_path)
    driver = webdriver.Chrome(service=s)
    driver.maximize_window()
    return driver

In [3]:
def scrape_reviews(driver, url):

    driver.get(url)
    time.sleep(5)

    reviews_button = driver.find_element(By.CSS_SELECTOR,'#reviews-medley-footer > div.a-row.a-spacing-medium > a')
    reviews_button.get_attribute('href')
    reviews_button.click()
    time.sleep(0.75)

    totals =  driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[4]/div").text
    total_ratings_match = re.search(r'([\d,]+) total ratings', totals)
    if total_ratings_match:

        total_ratings = int(total_ratings_match.group(1).replace(",", ""))  # Remove comma

        # Extract total reviews
        total_reviews_match = re.search(r'([\d,]+) with reviews', totals)
        if total_reviews_match:
            total_reviews = int(total_reviews_match.group(1).replace(",", ""))  # Remove comma

    n = round(total_reviews/10)
    print(n)
    star_rating = []
    review = []
    name = []
    review_title = []
    votes = []
    verified = []
    # ReviewID = []


    for j in range(n):
        review_elements = driver.find_elements(By.CSS_SELECTOR,'i[data-hook="review-star-rating"] span.a-icon-alt')
        page_total_reviews = len(review_elements)

        for i in range(1, page_total_reviews + 1):
            try:
                rating = driver.find_elements(By.CSS_SELECTOR,'i[data-hook="review-star-rating"] span.a-icon-alt')
                rating = rating[i-1].get_attribute('innerHTML')
                star = float(rating.split()[0])

                rev = driver.find_element(By.XPATH, f"/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i+1}]/div/div/div[4]/span/span").text
                rev_name = driver.find_element(By.XPATH, f"/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i+1}]/div/div/div[1]/a/div[2]/span").text
                title = driver.find_element(By.XPATH, f"/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i+1}]/div/div/div[2]/a/span[2]").text
                
                try: helpful_votes =  driver.find_element(By.XPATH, f"/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i+1}]/div/div/div[5]/div/span[1]/div[1]/span").text
                except: helpful_votes = '0 helpful votes'

                try: verified_purchase = driver.find_element(By.XPATH, f"/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i+1}]/div/div/div[3]/a[2]/span").text
                except: verified_purchase = 'Not a Verified Purchase'

            except Exception as e: print(e)

            star_rating.append(star)
            review.append(rev)
            name.append(rev_name)
            review_title.append(title)
            if helpful_votes == 'Helpful': votes.append('0 helpful votes')
            else: votes.append(helpful_votes)
            verified.append(verified_purchase)
        
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '#cm_cr-pagination_bar > ul > li.a-last > a'))
            )
            next_button.click()
            time.sleep(1.5)
        except:
            break
        print(f'{j} is done')
    return star_rating, review, name, review_title, votes, verified

In [4]:
def main():
    urls = ['https://www.amazon.com/dp/B0BYLJ3HH7?ref_=cm_sw_r_cp_ud_dp_K9518ZWP7QPY9CGBKXD3&th=1', 'https://www.amazon.com/dp/B09WZF9633?ref_=cm_sw_r_cp_ud_dp_4T7X35DYN1G52PX889BN_1&th=1',
            'https://www.amazon.com/dp/B08XYP6BJV?ref_=cm_sw_r_cp_ud_dp_FX17EPC24AF0YC1B2JX4_1', 'https://www.amazon.com/dp/B08M91FTJW?_encoding=UTF8&psc=1&ref_=cm_sw_r_cp_ud_dp_D4PC0KH8VM0MZ9RWQM8C',
            'https://www.amazon.com/dp/B07F3SNQT5?ref_=cm_sw_r_cp_ud_dp_YAM85QGN8A18TXN92KWV&th=1', 'https://www.amazon.com/dp/B0CLBV27N2?ref_=cm_sw_r_cp_ud_dp_G9F99MT1TGF7PC7D5HM6',
            'https://www.amazon.com/dp/B089TYDR5H?ref_=cm_sw_r_cp_ud_dp_KXGVA4KFVAT77G1J6PJ6', 'https://www.amazon.com/dp/B0007MZV3C?ref_=cm_sw_r_cp_ud_dp_BC28X8S1GE803Z5J2EMW',
            'https://www.amazon.com/dp/B0BVXDPZP3?ref_=cm_sw_r_cp_ud_dp_CM2YG40E4Y01Q2PRJ9N0', 'https://www.amazon.com/dp/B0074WGG82?ref_=cm_sw_r_cp_ud_dp_N977QKQM7GSCWK6MK4JP',
            'https://www.amazon.com/dp/B0CF3BDBL2?_encoding=UTF8&psc=1&ref_=cm_sw_r_cp_ud_dp_PV9BWYBQ750GN0H6V2YD', 'https://www.amazon.com/dp/B01MY05VGS?ref_=cm_sw_r_cp_ud_dp_FEJ1WCTP3XSGEH3FN3QQ',
            'https://www.amazon.com/dp/B0C5NT72GS?ref_=cm_sw_r_cp_ud_dp_ZSHAFJ2Z1K3KPFNT7NXH', 'https://www.amazon.com/dp/B07TCQN5NZ?ref_=cm_sw_r_cp_ud_dp_D77QM8FAK1MGRT0V5GFC',
            'https://www.amazon.com/dp/B0B9YSJC5S?ref_=cm_sw_r_cp_ud_dp_NYQDHQ80S7GSFRDBSJK4', 'https://www.amazon.com/dp/B0BPD4JMWT?ref_=cm_sw_r_cp_ud_dp_EHQGA0G67JAF1T05AKBR',
            'https://www.amazon.com/dp/B01N1037CV?ref_=cm_sw_r_cp_ud_dp_7FZB3R32WMX4K75G33ZE', 'https://www.amazon.com/dp/B0C6BQVDX3?ref_=cm_sw_r_cp_ud_dp_398G1X22ZGM8V9MB24XK', 
            'https://www.amazon.com/dp/B08YS7QZLL?_encoding=UTF8&psc=1&ref_=cm_sw_r_cp_ud_dp_ZH9B4M6GG711HES4BND6','https://www.amazon.com/dp/B0B57QV5TT?ref_=cm_sw_r_cp_ud_dp_5FH4Z31TY8GAA00ER85Y', 
            'https://www.amazon.com/dp/B0BSP2QC9F?ref_=cm_sw_r_cp_ud_dp_MWXX9YJN7E8S0VXHQTNA_1','https://www.amazon.com/dp/B0BHKQ45VR?ref_=cm_sw_r_cp_ud_dp_9GPNMYNJHEP1J76QXEWF',
            'https://www.amazon.com/dp/B097BQZFCW?ref_=cm_sw_r_cp_ud_dp_ZPPVC1X8E6ZCFYZC0GV2', 'https://www.amazon.com/dp/B08LJVMZD8?ref_=cm_sw_r_cp_ud_dp_TV7802KH8XE44QTCCB8M',]

    driver = initialize_driver('C:/Users/uttar/OneDrive/Desktop/Data Mining/chromedriver.exe')

    all_review_dfs = []

    for url in urls:
        star_rating, review, name, review_title, votes, verified = scrape_reviews(driver, url)
        review_df = pd.DataFrame({
            'review_header': review_title,
            'reviewer_name': name,
            'review_content': review,
            'review_star_rating': star_rating,
            'review_helpful_votes': votes,
            'verified_purchase': verified
        })

        all_review_dfs.append(review_df)

    driver.quit()

    return all_review_dfs

In [5]:
# Call the main function to get data for all URLs
all_review_dfs = main()

10
0 is done
1 is done
2 is done
3 is done
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[6]/div/div/div[4]/span/span"}
  (Session info: chrome=122.0.6261.113); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7F2DDAD22+56930]
	(No symbol) [0x00007FF7F2D4F622]
	(No symbol) [0x00007FF7F2C042E5]
	(No symbol) [0x00007FF7F2C498ED]
	(No symbol) [0x00007FF7F2C49A2C]
	(No symbol) [0x00007FF7F2C8A967]
	(No symbol) [0x00007FF7F2C6BCDF]
	(No symbol) [0x00007FF7F2C881E2]
	(No symbol) [0x00007FF7F2C6BA43]
	(No symbol) [0x00007FF7F2C3D438]
	(No symbol) [0x00007FF7F2C3E4D1]
	GetHandleVerifier [0x00007FF7F3156AAD+3709933]
	GetHandleVerifier [0x00007FF7F31AFFED+4075821]
	GetHandleVerifier [0x00007FF7F31A817F+4043455]
	GetHandleVerifier [0x00007FF7F2E79756+706710]
	

In [6]:
all_review_dfs

[                             review_header                 reviewer_name  \
 0   Wonderful product and would recommend!                           S&K   
 1        It is renewed but looked like new                         Andre   
 2                                 Worth it                         Alice   
 3                      Brick Stolen Device                     Dana Rajc   
 4                                  Minimum               Dennis Deckmann   
 ..                                     ...                           ...   
 95                This is a waste of money                   Andrea Ogle   
 96                            As expected!                    Colin Reed   
 97                                      🤦🏻                       Valerie   
 98                            Doesn’t work                    Brigitte V   
 99                        Excelente compra  Luis Daniel Lezama Galaschow   
 
                                        review_content  review_star_rating

In [7]:
dfs = []

for i, df in enumerate(all_review_dfs):
    df.insert(0, 'productID', i+1)
    df.insert(1, 'prodSiteID', f"Amazon{i+1}")
    df.insert(2, 'ReviewID', range(1, len(df) + 1))
    dfs.append(df)
    #df.to_excel(f'../data/amazon_url_{i+1}_review_data.xlsx', index=False)
concatenated_df = pd.concat(dfs, ignore_index=True)

concatenated_df.to_excel("../data/Amazon_Review_data_all.xlsx", index=False)
print("Concatenation complete. New Excel file created.")

Concatenation complete. New Excel file created.


In [8]:
import re

# Read the Excel file
df = pd.read_excel('../data/Amazon_Review_data_all.xlsx')

# Function to extract the number of helpful votes
def extract_helpful_votes(text):
    first_word = text.split()[0]
    if first_word.isdigit():
        return int(first_word)
    elif first_word.lower() == 'one':
        return 1
    else:
        return 0

# Apply the function to the 'Helpful_votes' column
df['review_helpful_votes'] = df['review_helpful_votes'].apply(extract_helpful_votes)

df['verified_purchase'] = df['verified_purchase'].apply(lambda x: True if x == 'Verified Purchase' else False)

# Save the modified DataFrame back to Excel
df.to_excel('../data/Amazon_Review_data_all.xlsx', index=False)

print("Edits complete. Excel file overwritten.")

Edits complete. Excel file overwritten.
